### **Libraries and Dependencies**

In [1]:
import os
import tempfile
import whisper
from groq import Groq
from gtts import gTTS
import gradio as gr
from IPython.display import Audio, display

c:\1BAU\7 SEMESTER\Special topic I\ASIGGN\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **Loading Whisper Speech Recognition Model**

In [2]:
print("Loading Whisper model...")
whisper_model = whisper.load_model("base")
print("Whisper model loaded!")

Loading Whisper model...
Whisper model loaded!


### **Groq API Configuration**

In [3]:
GROQ_API_KEY = "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"

### **Core System Functions**

In [4]:
def speech_to_text(audio_path: str) -> str:
    if audio_path is None:
        return "No audio file provided."
    result = whisper_model.transcribe(audio_path)
    return result["text"]

def generate_answer(question: str, api_key: str) -> str:
    if not api_key:
        return "Please provide a Groq API key."
    if not question or question == "No audio file provided.":
        return "No valid question to answer."
    
    try:
        client = Groq(api_key=api_key)
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Answer questions clearly and concisely."},
                {"role": "user", "content": question}
            ],
            max_tokens=500
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating answer: {str(e)}"


def text_to_speech(text: str) -> str:
    if not text or text.startswith("Error") or text.startswith("Please provide") or text.startswith("No valid"):
        return None
    try:
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        temp_path = temp_file.name
        temp_file.close()
        tts = gTTS(text=text, lang='en')
        tts.save(temp_path)
        return temp_path
    except Exception as e:
        print(f"TTS Error: {e}")
        return None

def process_voice_question(audio_file, api_key: str):
    question_text = speech_to_text(audio_file)
    answer_text = generate_answer(question_text, api_key)
    answer_audio = text_to_speech(answer_text)
    return question_text, answer_text, answer_audio

print("Functions defined!")

Functions defined!


### **Manual Audio Processing**

In [5]:
audio_file_path = "sample_question.wav"
print("Processing audio question...")

print("\nStep 1: Converting speech to text...")
question = speech_to_text(audio_file_path)
print(f"Question: {question}")

print("\nStep 2: Generating answer with Groq LLM...")
answer = generate_answer(question, GROQ_API_KEY)
print(f"Answer: {answer}")

print("\nStep 3: Converting answer to speech...")
audio_output = text_to_speech(answer)
print(f"Audio saved to: {audio_output}")

if audio_output:
    print("\nPlaying answer audio:")
    display(Audio(audio_output))


Processing audio question...

Step 1: Converting speech to text...


c:\1BAU\7 SEMESTER\Special topic I\ASIGGN\.venv\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Question:  What is artificial intelligence and how does it work?

Step 2: Generating answer with Groq LLM...
Answer: **Artificial Intelligence (AI)**: Artificial intelligence refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception.

**How it works**: AI works by using algorithms and data to enable machines to make decisions and take actions. The process involves:

1. **Data collection**: Gathering data relevant to the task or problem.
2. **Data processing**: Using algorithms to analyze and process the data.
3. **Learning**: Identifying patterns and relationships in the data to improve performance.
4. **Decision-making**: Using the learned patterns to make decisions or predictions.
5. **Action**: Taking actions based on the decisions made.

Types of AI include:

* **Narrow or Weak AI**: Designed to perform a specific task, such as image recognition or language trans

### **Interactive Web Interface (Gradio)**

In [6]:
def create_interface():
    with gr.Blocks(title="Voice Q&A System") as demo:
        
        with gr.Row():
            with gr.Column():
                api_key_input = gr.Textbox(
                    label=" Groq API Key",
                    value=GROQ_API_KEY,
                    type="password"
                )
                audio_input = gr.Audio(
                    label=" Upload Your Question",
                    type="filepath",
                    sources=["upload", "microphone"]
                )
                submit_btn = gr.Button(" Get Answer", variant="primary")

        with gr.Row():
            transcription_output = gr.Textbox(label="Transcribed Question", lines=2)
            answer_output = gr.Textbox(label="Generated Answer", lines=4)
        
        audio_output = gr.Audio(label="Answer (Audio)", type="filepath")
        submit_btn.click(
            fn=process_voice_question,
            inputs=[audio_input, api_key_input],
            outputs=[transcription_output, answer_output, audio_output]
        )
    return demo
demo = create_interface()
demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\Next-Level\AppData\Local\Programs\Python\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Next-Level\AppData\Local\Programs\Python\Python310\lib\asyncio\proactor_events.py", line 162, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
c:\1BAU\7 SEMESTER\Special topic I\ASIGGN\.venv\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Trace